best tuned settings:
    *  R2 for test grs data: 0.6916471359322971

    *  negMedAE for test grs data: -10158.809876327752

    *  R2 for test gcs data: 0.20898394205704673

    *  negMedAE for test gcs data: -10157.19525109193

In [1]:
from helpers import utils
from os.path import join, dirname
from dotenv import load_dotenv
import os
import pickle
from snowflake import connector
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.preprocessing import Normalizer, QuantileTransformer, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from tpot.builtins import StackingEstimator, ZeroCount
from xgboost import XGBRegressor
from tpot.export_utils import set_param_recursive
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error, make_scorer


pd.options.display.float_format = '{:,.2f}'.format

# get environment variables
dotenv_path = join(dirname('streamlit_grs_fit\\app\\'), '.env')
load_dotenv(dotenv_path)
SF_ACCOUNT = os.getenv('SF_ACCOUNT')
SF_USER = os.getenv('SF_USER')
SF_PASSWORD = os.getenv('SF_PASSWORD')
SF_ROLE = os.getenv('SF_ROLE')
SF_WAREHOUSE = os.getenv('SF_WAREHOUSE')
SF_DATABASE = os.getenv('SF_DATABASE')
SF_SCHEMA = os.getenv('SF_SCHEMA')

def load_data(query):
    conn = connector.connect(
        user = SF_USER
        ,password = SF_PASSWORD
        ,account = SF_ACCOUNT
        ,warehouse = SF_WAREHOUSE
        ,database = SF_DATABASE
        ,schema = SF_SCHEMA
        ,role = SF_ROLE
    )
    cur = conn.cursor()
    df_data = cur.execute(query).fetch_pandas_all()
    return df_data

c:\Users\SwanS\Anaconda3\envs\grs_fit\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
query = 'select '+\
            'JOB'+\
            ',DIRECT_COST'+\
            ',DIV_00_DIRECT_COST'+\
            ',DIV_01_DIRECT_COST'+\
            ',DIV_02_DIRECT_COST'+\
            ',DIV_03_DIRECT_COST'+\
            ',DIV_04_DIRECT_COST'+\
            ',DIV_05_DIRECT_COST'+\
            ',DIV_06_DIRECT_COST'+\
            ',DIV_07_DIRECT_COST'+\
            ',DIV_08_DIRECT_COST'+\
            ',DIV_09_DIRECT_COST'+\
            ',DIV_10_DIRECT_COST'+\
            ',DIV_11_DIRECT_COST'+\
            ',DIV_12_DIRECT_COST'+\
            ',DIV_13_DIRECT_COST'+\
            ',DIV_14_DIRECT_COST'+\
            ',DIV_15_DIRECT_COST'+\
            ',DIV_16_DIRECT_COST'+\
            ',DIV_17_DIRECT_COST'+\
            ',DIV_18_DIRECT_COST'+\
            ',DIV_19_DIRECT_COST'+\
            ',DIV_21_DIRECT_COST'+\
            ',DIV_22_DIRECT_COST'+\
            ',DIV_23_DIRECT_COST'+\
            ',DIV_26_DIRECT_COST'+\
            ',DIV_27_DIRECT_COST'+\
            ',DIV_28_DIRECT_COST'+\
            ',DIV_31_DIRECT_COST'+\
            ',DIV_32_DIRECT_COST'+\
            ',DIV_33_DIRECT_COST'+\
            ',DIV_34_DIRECT_COST'+\
            ',DIV_55_DIRECT_COST'+\
            ',GCS_COST'+\
            ',GRS_COST '+\
            'from sandbox.global.ml_grs_fit ' 
df_data = load_data(query).set_index('JOB') 
df_data = pd.DataFrame(df_data)
df_data = df_data.fillna(0)

In [3]:
df_working = df_data.loc[
                    (0 != df_data.GRS_COST) &
                    (0 != df_data.GCS_COST)
].copy()
df_working.describe()

,DIRECT_COST,DIV_00_DIRECT_COST,DIV_01_DIRECT_COST,DIV_02_DIRECT_COST,DIV_03_DIRECT_COST,DIV_04_DIRECT_COST,DIV_05_DIRECT_COST,DIV_06_DIRECT_COST,DIV_07_DIRECT_COST,DIV_08_DIRECT_COST,...,DIV_26_DIRECT_COST,DIV_27_DIRECT_COST,DIV_28_DIRECT_COST,DIV_31_DIRECT_COST,DIV_32_DIRECT_COST,DIV_33_DIRECT_COST,DIV_34_DIRECT_COST,DIV_55_DIRECT_COST,GCS_COST,GRS_COST
count,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00",...,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00"
mean,"2,753,305.18","15,416.14","1,095.30","120,305.74","305,087.19","25,975.15","192,784.27","83,875.74","97,562.86","262,156.35",...,"279,623.43","22,192.54","7,159.77","84,671.51","18,546.22","19,750.99",505.83,"1,710.05","159,975.26","130,762.61"
std,"18,205,353.59","247,803.94","21,856.71","835,518.53","2,456,075.21","255,679.11","1,926,522.10","479,208.15","712,596.43","2,347,850.24",...,"2,283,423.57","355,302.65","98,050.42","852,341.42","200,217.24","315,920.99","14,397.44","98,425.42","991,159.86","893,014.97"
min,"-3,404,153.00","-327,264.32",0.00,"-41,282.56","-4,315,458.20","-164,664.12","-1,398,362.17","-174,550.72","-16,197.24","-407,656.41",...,-104.77,0.00,0.00,"-2,322.10",-11.50,"-53,330.53",-4.20,0.00,"-3,335,087.00","-629,785.00"
25%,"4,410.75",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,554.25,305.75
50%,"31,937.00",0.00,0.00,889.22,0.00,0.00,0.00,353.55,0.00,37.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"4,763.00","2,099.00"
75%,"279,720.25",0.00,0.00,"10,486.36","1,041.34",0.00,180.50,"10,992.53",44.95,"9,230.26",...,"3,893.87",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"35,369.50","16,783.00"
max,"399,570,438.00","8,618,727.16","762,087.69","26,715,320.64","49,560,386.64","9,635,075.36","51,850,542.49","9,711,287.61","19,337,427.69","56,887,683.67",...,"57,604,860.12","14,904,687.98","2,706,150.04","19,786,153.90","6,898,015.60","10,785,799.69","607,500.00","5,681,453.04","23,749,853.00","18,182,714.00"


In [4]:
X  = df_working.iloc[:,:-2] #.values
y_gcs = df_working.iloc[:,-2:-1].values.ravel()
y_grs = df_working.iloc[:,-1:].values.ravel()
y_grs.shape

(3332,)

In the last itteration, I got everything I was trying to do to work. that is, I trained a model and saved it as a binary object that can be packaged with an app, and tested the implementation within the app framework. 
that part all works great! it's super fast, and I think we'll be able to get the ui pretty nice at the end.
the thing to be aware of at this point is that it can't be expected to be reliable. yet
here's what I think are next steps in order to make it as reliable as possible, and to be able to know exactly how reliable it is. this is what I'll do in this itteration:
1.  identify metrics by which model success will be measured. e.g., MSE, RMSE, MAE, MAPE
2.  implement train / test split prior to cross validation
3.  rerun the cross validation setup for this estimator (knearestneighborsregressor) with the training data only
4.  evaluate, in terms of our selected metrics, the model selected by cross validation against the test data (to which it would be naïve at this point. this will guard us against data leakage)
5.  repeat steps 3 and 4 for a few other estimators. I'm thinking lasso, elasticnet, maybe ridgeregression or svr, and perhaps mlpregressor if we're feeling ambitious
compare the scores of our models and pick the best one

initially, trying these scorers: r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error

In [5]:
X_train, X_test, y_grs_train, y_grs_test = train_test_split(X, y_grs, test_size=0.33, random_state=42)
X_train, X_test, y_gcs_train, y_gcs_test = train_test_split(X, y_gcs, test_size=0.33, random_state=42)

In [6]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# pipeline setup
pipeline = Pipeline([
                     ('scaler', None)
                    #  ,('kbest', SelectKBest(f_classif))
                     ,('z_count', ZeroCount())
                     ,('stack_reg', StackingEstimator(estimator=XGBRegressor(n_estimators=100, objective="reg:squarederror", subsample=0.8500000000000001, verbosity=0)))
                     ,('meta_reg', ElasticNetCV(l1_ratio=0.1, tol=0.001))
                     ])

parameters = {
                # 'scaler':  [RobustScaler(), Normalizer(), QuantileTransformer()]
                # ,'kbest__k':  list(range(1, X.shape[1]+1))
                'stack_reg__estimator__learning_rate': np.linspace(0.4, 0.6, 3)
                ,'stack_reg__estimator__max_depth': [int(_) for _ in np.linspace(4, 6, 3)]
                ,'stack_reg__estimator__min_child_weight': [int(_) for _ in np.linspace(3, 5, 3)]
                ,'meta_reg__l1_ratio':  np.linspace(0.05, 0.15, 3)
                ,'meta_reg__tol':  np.linspace(0.0005, 0.0015, 3)
                }
#grs model
grs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=cv
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(utils.neg_median_absolute_error)
    }
    # ,refit='R2'
    ,refit=utils.refit_strategy
    ,return_train_score=False
    ,n_jobs=-2
)   

#gcs model
gcs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=cv
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(utils.neg_median_absolute_error)
        }
    ,refit=utils.refit_strategy
    ,return_train_score=False
    ,n_jobs=-2
)   

In [ ]:
[int(_) for _ in np.linspace(4, 6, 3)]

In [7]:
grs_grid = grs_grid.fit(X_train, y_grs_train)

All grid-search results:
R2: 0.706 (±0.371), negMedAE: -15244.269 (±2190.955), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 3}
R2: 0.703 (±0.362), negMedAE: -15299.524 (±2187.276), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 4}
R2: 0.733 (±0.306), negMedAE: -15406.763 (±2112.304), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 5}
R2: 0.708 (±0.363), negMedAE: -15141.612 (±2164.200), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 5, 'stack_reg__estimator__min_child_weight': 3}
R2: 0.703 (±0.356),

In [9]:
gcs_grid = gcs_grid.fit(X_train, y_gcs_train)

All grid-search results:
R2: 0.616 (±0.286), negMedAE: -18199.173 (±1783.434), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 3}
R2: 0.535 (±0.390), negMedAE: -17996.270 (±1875.717), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 4}
R2: 0.489 (±0.416), negMedAE: -18160.537 (±1868.838), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 4, 'stack_reg__estimator__min_child_weight': 5}
R2: 0.627 (±0.296), negMedAE: -18472.095 (±1446.598), for {'meta_reg__l1_ratio': 0.05, 'meta_reg__tol': 0.0005, 'stack_reg__estimator__learning_rate': 0.4, 'stack_reg__estimator__max_depth': 5, 'stack_reg__estimator__min_child_weight': 3}
R2: 0.537 (±0.347),

In [10]:
y_grs_test_pred = grs_grid.best_estimator_.predict(X_test)
print(f'R2 for test grs data: {r2_score(y_grs_test, y_grs_test_pred)}')
print(f'negMedAE for test grs data: {utils.neg_median_absolute_error(y_grs_test, y_grs_test_pred)}')

R2 for test grs data: 0.6916471359322971
negMedAE for test grs data: -10158.809876327752


In [11]:
y_gcs_test_pred = gcs_grid.best_estimator_.predict(X_test)
print(f'R2 for test gcs data: {r2_score(y_gcs_test, y_gcs_test_pred)}')
print(f'negMedAE for test gcs data: {utils.neg_median_absolute_error(y_gcs_test, y_gcs_test_pred)}')

R2 for test gcs data: 0.20898394205704673
negMedAE for test gcs data: -10157.19525109193


In [12]:
gcs_grid_test = gcs_grid.best_estimator_.fit(X_test, y_gcs_test)

In [13]:
print("the best grs estimator is \n {} ".format(grs_grid.best_estimator_))
print("the best grs parameters are \n {}".format(grs_grid.best_params_))
print("the best gcs estimator is \n {} ".format(gcs_grid.best_estimator_))
print("the best gcs parameters are \n {}".format(gcs_grid.best_params_))

the best grs estimator is 
 Pipeline(steps=[('scaler', None), ('z_count', ZeroCount()),
                ('stack_reg',
                 StackingEstimator(estimator=XGBRegressor(base_score=None,
                                                          booster=None,
                                                          callbacks=None,
                                                          colsample_bylevel=None,
                                                          colsample_bynode=None,
                                                          colsample_bytree=None,
                                                          device=None,
                                                          early_stopping_rounds=None,
                                                          enable_categorical=False,
                                                          eval_metric=None,
                                                          feature_types=None,
                      

In [ ]:
grs_best_pipe = grs_grid.best_estimator_
grs_mask = list(grs_best_pipe.fit(X,y_grs)[:-1].get_feature_names_out())
grs_model = grs_best_pipe.fit(df_working[grs_mask],y_grs)
grs_predictions = grs_model.predict(df_working[grs_mask])

In [ ]:
gcs_best_pipe = gcs_grid.best_estimator_
gcs_mask = list(gcs_best_pipe.fit(X,y_gcs)[:-1].get_feature_names_out())
gcs_model = gcs_best_pipe.fit(df_working[gcs_mask],y_gcs)
gcs_predictions = gcs_model.predict(df_working[gcs_mask])

In [ ]:
list(grs_model[:-1].get_feature_names_out())

In [ ]:
grs_parameters = list(df_working[grs_mask].columns)
gcs_parameters = list(df_working[gcs_mask].columns)
combined_mask = list(set(grs_parameters + gcs_parameters))
df = df_working[combined_mask].copy()
df['GRS_PREDICTIONS'] = grs_predictions
df['GCS_PREDICTIONS'] = gcs_predictions
knnr_model_bag = {
    'df': df
    ,'grs_model': grs_model
    ,'grs_parameters': grs_parameters
    ,'gcs_model': gcs_model
    ,'gcs_parameters': gcs_parameters
}
with open('./app/knnr_model_bag.pkl','wb') as p:
    pickle.dump(knnr_model_bag, p, protocol=-1)

In [ ]:
with open('./app/model_bag.pkl','rb') as p:
    bag = pickle.load(p)

In [ ]:
bag.keys()

In [ ]:
grs_params = bag['grs_parameters']
gcs_params = bag['gcs_parameters']
all_params = list(set(grs_params + gcs_params))
test_vec = bag['df'][all_params].sample(1).copy()
# bag['grs_model'].predict(test_vec)
# model = bag['grs_model']
# list(model[:-1].get_feature_names_out())
print(*list(test_vec[grs_params].columns), sep='\n,')

In [ ]:
vec = test_vec.reset_index(drop=True).T
vec.index.names = ['PARAMETERS']
vec = vec.reset_index()
vec.set_index('PARAMETERS').sort_index()

In [ ]:
bag['grs_model'].predict(test_vec)

In [ ]:
# get the features scores rounded in 2 decimals
pip_steps = grs_grid.best_estimator_.named_steps['kbest']
pip_steps.get_support()

features_scores = ['%.2f' % elem for elem in pip_steps.scores_ ]
print("the features scores are \n {}".format(features_scores))

feature_scores_pvalues = ['%.3f' % elem for elem in pip_steps.pvalues_]
print("the feature_pvalues is \n {} ".format(feature_scores_pvalues))

scored_features = pd.DataFrame(df_working[grs_mask].columns, columns=['feature_names'])
scored_features['feature_scores'] = features_scores
scored_features['feature_scores_pvalues'] = feature_scores_pvalues
scored_features = scored_features.loc[(scored_features['feature_scores'] != 'nan') & (scored_features['feature_scores'] != 'inf')].sort_values(by='feature_scores', ascending=False).iloc[:num_features]
scored_features

In [ ]:
selected_features = scored_features.feature_names.to_list()
df_working[selected_features].describe()

In [ ]:
pickle.dump(neigh, open('grs_model.pkl','wb'))

In [ ]:
grs_model = pickle.load(open('grs_model.pkl','rb'))

In [ ]:
pickle.dump(data_preds, open('grs_model.pkl','ab+'))

In [ ]:
grs_data = []
with open('./app/grs_model.pkl', 'rb') as fr:
    try:
        while True:
            grs_data.append(pickle.load(fr))
    except EOFError:
        pass
gcs_data = []
with open('./app/gcs_model.pkl', 'rb') as fr:
    try:
        while True:
            gcs_data.append(pickle.load(fr))
    except EOFError:
        pass

In [ ]:
grs_model, grs_preds = grs_data
gcs_model, gcs_preds = gcs_data

In [ ]:
graphWidth = 1500
graphHeight = graphWidth * 800 / 1000
x_plot = data_preds.DIRECT_COST
y1_plot = data_preds.GRS_ACTUAL
y2_plot = data_preds.GRS_PREDICTIONS
f = plt.figure(figsize=(graphWidth/100.0, graphHeight/100.0), dpi=100)
axes = f.add_subplot(111)
axes.plot(x_plot, y1_plot, c='g', alpha=0.15)
axes.plot(x_plot, y2_plot, alpha=0.15)
axes.scatter(direct_cost, grs_cost, c='r', marker='D')
plt.show()